In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from tensorflow import keras

In [ ]:
print("GPU Available: ", tf.test.is_gpu_available())
print("Eager execution enabled: ", tf.executing_eagerly())

# Load and rescale data

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

scale = tf.constant(255, dtype=tf.dtypes.float32)
x_train, x_test = train_images/scale, test_images/scale
y_train, y_test = tf.expand_dims(train_labels, 1), tf.expand_dims(test_labels, 1)

# Define and compile

In [ ]:
# single dense layer, i.e. multiple logistic regression
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

training_params = {'optimizer': 'adam',
                   'loss': 'sparse_categorical_crossentropy',
                   'metrics': ['accuracy']}

tf.random.set_seed(123)
np.random.seed(123)
model.compile(**training_params)

model.summary()

In [ ]:
# run the training
tensorboard_callback = keras.callbacks.TensorBoard()
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

# Convert to TFLite and save to disk

In [ ]:
models_dir = pathlib.Path("./quantizer_tmp_models/")
models_dir.mkdir(exist_ok=True, parents=True)

### Float TFLite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_float_lite = converter.convert()

In [ ]:
model_float_file = models_dir/"model_float.tflite"
size_float = model_float_file.write_bytes(model_float_lite)
print('Float model size: {:.0f} KB'.format(size_float/1024))

### Quantized TFLite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # this doesn't seem to do anything

# representative dataset to estimate activation distributions
x_train_ds = tf.data.Dataset.from_tensor_slices((x_train)).batch(1)
def representative_data_gen():
    for input_value in x_train_ds.take(100):
        yield [input_value]
converter.representative_dataset = representative_data_gen

model_quant_lite = converter.convert()

In [ ]:
model_quant_file = models_dir/"model_quant.tflite"
size_quant = model_quant_file.write_bytes(model_quant_lite)
print('Quantized model size: {:.0f} KB'.format(size_quant/1024))

### Build interpreters

In [ ]:
interpreter_float = tf.lite.Interpreter(model_content=model_float_lite)
interpreter_float.allocate_tensors()
interpreter_quant = tf.lite.Interpreter(model_content=model_quant_lite)
interpreter_quant.allocate_tensors()

# Interpreter surgery

In [ ]:
# run interpreters on a single sample
img = tf.expand_dims(x_test[10], 0)
interpreter_float.set_tensor(interpreter_float.get_input_details()[0]["index"], img)
interpreter_float.invoke()
interpreter_quant.set_tensor(interpreter_quant.get_input_details()[0]["index"], img)
interpreter_quant.invoke()